### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
# !wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
# !tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
30233,"[{'name': 'Lei Liu'}, {'name': 'Zongxu Pan'}, ...",26,1711.09405v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Detection of arbitrarily rotated objects is a ...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Learning a Rotation Invariant Detector with Ro...,2017
33483,"[{'name': 'Feng Gu'}, {'name': 'Julie Greensmi...",31,1305.7476v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,As one of the emerging algorithms in the field...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Theoretical formulation and analysis of the de...,2013
32820,"[{'name': 'Narayanan U. Edakunni'}, {'name': '...",13,1508.03130v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,Research in transportation frequently involve ...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Probabilistic Dependency Networks for Predicti...,2015
21693,"[{'name': 'Finn Macleod'}, {'name': 'James Gle...",10,0812.1949v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,We obtain an index of the complexity of a rand...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Prediction with Restricted Resources and Finit...,2008
25725,"[{'name': 'Devendra Kumar Sahu'}, {'name': 'Mo...",13,1511.04176v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,We propose an end-to-end recurrent encoder-dec...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Sequence to Sequence Learning for Optical Char...,2015


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [4]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer

import nltk

tokenizer = nltk.tokenize.WordPunctTokenizer()


def _tokenize(text:str) -> str:
    text = str(text)
    return ' '.join(tokenizer.tokenize(text)).lower()

for i in range(len(lines)):
    lines[i] = _tokenize(lines[i])


In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [6]:
# special tokens: 
# - `UNK` represents absent tokens, 
# - `EOS` is a special token after the end of sequence

from tqdm import tqdm
from collections import defaultdict, Counter

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)

    for line in tqdm(lines): 
        words = line.split()
        words.append(EOS)  # Добавляем специальный токен EOS в конец каждой строки
        words = [UNK] * (n - 1) + words  # Добавляем UNK перед началом строки для первой n-граммы

        # Перебираем слова для создания n-грамм
        for i in range(len(words) - n + 1):
            prefix = tuple(words[i:i + n - 1])  # Создаем префикс из (n - 1) слов
            next_word = words[i + n - 1]  # Получаем следующее слово после префикса
            counts[prefix][next_word] += 1  # Обновляем счетчик для данной n-граммы
            
    for prefix, counter in counts.items():
        counts[prefix] = dict(counter)        
    return counts


In [7]:
l = ['I go to the school. mother and father love me mother go mother no']

In [8]:
count_ngrams(l,2)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


defaultdict(collections.Counter,
            {('_UNK_',): {'I': 1},
             ('I',): {'go': 1},
             ('go',): {'to': 1, 'mother': 1},
             ('to',): {'the': 1},
             ('the',): {'school.': 1},
             ('school.',): {'mother': 1},
             ('mother',): {'and': 1, 'go': 1, 'no': 1},
             ('and',): {'father': 1},
             ('father',): {'love': 1},
             ('love',): {'me': 1},
             ('me',): {'mother': 1},
             ('no',): {'_EOS_': 1}})

In [9]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 36204.61it/s]


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [10]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix, token_counter in counts.items():
            count_sum = sum(token_counter.values())
            for token, token_count in token_counter.items():
                self.probs[prefix][token] = token_count / count_sum

            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [11]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 33341.05it/s]


Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [12]:
lm = NGramLanguageModel(lines, n=3)

100%|██████████████████████████████████████████████████████████████████████████| 41000/41000 [00:08<00:00, 4573.14it/s]


In [13]:
prefix = "the gnn"

print(end=prefix + ' ', flush=True)
for i in range(200):
    tokens, probs = zip(*lm.get_possible_next_tokens(prefix).items())
    next_token = np.random.choice(tokens, p=probs)
    prefix = prefix + ' ' + next_token
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
    print(end=next_token + ' ', flush=True)

the gnn is a meta classifier that effectively and efficiently improve the effectiveness and efficiency of our method outperforms both active learning or other optimization techniques . analytically , rather than reasoning about 2d segments into one learning step and propose several methods exist for estimating parameters in an immutable manner . it works on paper is the one hand , manual tweaking of learning . we study abduction in abstract argumentation frameworks ; over the baseline with moderate amounts of ( database ) are nonparametric analogs of linguistic subsystems simultaneously and n is the cost - effectively by training parameters than other previous works for any positive constant $\ epsilon $- accurate solution does not overtrain easily . the subject ' s two known shortcomings of current research in computer vision based applications need to take advantage of both architectures , of high - performance deep learning may be complex . this allows us to invoke properties of nad

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [14]:
# def get_next_token(lm, prefix, temperature=1.0):
#     """
#     return next token after prefix;
#     :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
#         if temperature == 0, always takes most likely token. Break ties arbitrarily.
#     """
#     print(end=prefix + ' ', flush=True)
#     while True:  # Заменено на бесконечный цикл
#         next_tokens_dict = lm.get_possible_next_tokens(prefix)
#         if not next_tokens_dict:  # Если нет следующих токенов, завершаем генерацию
#             break

#         tokens, probs = zip(*next_tokens_dict.items())

#         if temperature == 0:
#             next_token = tokens[np.argmax(probs)]
#         else:
#             probs = np.array(probs)**(1/temperature)
#             probs /= probs.sum()
#             next_token = np.random.choice(tokens, p=probs)

#         if next_token == EOS:  # Проверка на специальный токен конца последовательности
#             break

#         prefix = prefix + ' ' + next_token
#         print(end=next_token + ' ', flush=True)

# # Использование функции
# get_next_token(lm=lm, prefix="the gnn is", temperature=1.3)


In [15]:
import numpy as np

def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    next_tokens_dict = lm.get_possible_next_tokens(prefix)
    if not next_tokens_dict:
        return None  # Если нет следующих токенов

    tokens, probs = zip(*next_tokens_dict.items())

    if temperature == 0:
        # Возвращает самый вероятный токен
        return tokens[np.argmax(probs)]
    else:
        probs = np.array(probs)**(1 / temperature)
        probs /= probs.sum()  # Нормализация вероятностей
        return np.random.choice(tokens, p=probs)  # Возвращает следующий токен

# Этот код следует запускать отдельно для каждого тестового случая.

        
get_next_token(lm=lm, prefix="there have", temperature=2)

'been'

In [16]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [17]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial neural networks . recently , substantial computation can be applied to sms messages that are difficult to obtain multi - objects intermediates ; the separability assumption , which may be able to answer two questions : 1 ) estimates the mav to adjust , having large infeasible regions . joint learning process , standard errors , providing theoretical results are obtained and convergence performance on challenging document and generating revenue however , their design is very useful and full groups of nodes upper and lower errors and naming chronometry research traditions that have been extensively studied ica algorithms using this


In [18]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two - fold cross validation method . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [19]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    log_prob_sum = 0
    token_count = 0
    
    for line in lines:
        words = line.split() + [EOS]  # Добавляем EOS в конец каждой строки
        for i in range(len(words)):
            prefix = " ".join(words[max(0, i - lm.n + 1):i])
            next_word = words[i]
            
            log_prob_sum += np.log(max(lm.get_next_token_prob(prefix, next_word), np.exp(min_logprob)))
            
            token_count += 1
#         print(log_prob_sum) 
#         print(token_count)
            
    return np.exp(-log_prob_sum / token_count)

In [20]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
# assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 33117.28it/s]

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [21]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


100%|█████████████████████████████████████████████████████████████████████████| 30750/30750 [00:01<00:00, 15624.98it/s]


N = 1, Perplexity = 1832.23136


100%|██████████████████████████████████████████████████████████████████████████| 30750/30750 [00:03<00:00, 9462.40it/s]


N = 2, Perplexity = 85653987.28774


100%|██████████████████████████████████████████████████████████████████████████| 30750/30750 [00:06<00:00, 4719.74it/s]


N = 3, Perplexity = 61999196259043346743296.00000


In [22]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [23]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
            
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        print(token_probs)
        missing_prob_total = 1.0 - sum(token_probs.values())
        print(missing_prob_total)
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [24]:
dummy_lm = LaplaceLanguageModel(dummy_lines, n=2)

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 50003.62it/s]


In [25]:
dummy_lm.get_possible_next_tokens('is')

{'a': 0.005107252298263534, 'np': 0.0020429009193054137, 'shown': 0.0030643513789581204, 'taken': 0.0020429009193054137, 'proposed': 0.0030643513789581204, 'demonstrated': 0.0020429009193054137, 'no': 0.0030643513789581204, 'arxiv': 0.0020429009193054137, 'presented': 0.0030643513789581204, 'written': 0.0020429009193054137, 'the': 0.0040858018386108275, 'not': 0.005107252298263534, 'discussed': 0.0020429009193054137, 'syllabic': 0.0020429009193054137, 'somewhat': 0.0020429009193054137, 'complete': 0.0020429009193054137, 'intentionally': 0.0020429009193054137, 'in': 0.0020429009193054137, 'raised': 0.0020429009193054137, 'to': 0.0030643513789581204}
0.9458631256384065


{'metaheuristics': 0.0010214504596527069,
 'briefly': 0.0010214504596527069,
 'their': 0.0010214504596527069,
 'homage': 0.0010214504596527069,
 'quantum': 0.0010214504596527069,
 'contrastive': 0.0010214504596527069,
 '0711': 0.0010214504596527069,
 'arxiv': 0.0020429009193054137,
 'with': 0.0010214504596527069,
 'variant': 0.0010214504596527069,
 'sets': 0.0010214504596527069,
 'used': 0.0010214504596527069,
 'on': 0.0010214504596527069,
 'other': 0.0010214504596527069,
 'realistic': 0.0010214504596527069,
 'paracontracting': 0.0010214504596527069,
 'dual': 0.0010214504596527069,
 'role': 0.0010214504596527069,
 'first': 0.0010214504596527069,
 'level': 0.0010214504596527069,
 'it': 0.0010214504596527069,
 'collected': 0.0010214504596527069,
 'handout': 0.0010214504596527069,
 'history': 0.0010214504596527069,
 'i': 0.0010214504596527069,
 'article': 0.0010214504596527069,
 'convex': 0.0010214504596527069,
 'more': 0.0010214504596527069,
 'evolution': 0.0010214504596527069,
 'cong': 

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [26]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 23567.48it/s]


In [27]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

100%|█████████████████████████████████████████████████████████████████████████| 30750/30750 [00:01<00:00, 15680.14it/s]


N = 1, Perplexity = 977.67559


100%|██████████████████████████████████████████████████████████████████████████| 30750/30750 [00:03<00:00, 9120.37it/s]


N = 2, Perplexity = 470.48021


100%|██████████████████████████████████████████████████████████████████████████| 30750/30750 [00:06<00:00, 4569.42it/s]


N = 3, Perplexity = 3679.44765
